not done.

the model can't predict never seen data.

In [3]:
import numpy as np
from scipy.stats import skew

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

In [2]:
# expected = 0
print(skew(a= [[-1],
               [0],
               [1]],
           axis=0))

[0.]


In [7]:
#              X1    y
X = np.array([
              [0], # 0
              [0], # 0
              [0], # 0
              [1], # 1
              [1], # 1
              [1], # 1
              [2], # 0
              [2], # 0
              [2], # 0
              [3], # 1
              [3], # 1
              [3], # 1
              [4], # 0
              [4], # 0
              [4], # 0
              [5], # 1
              [5], # 1
              [5], # 1
              [6], # 0
              [6], # 0
              [6], # 0
              [7], # 1
              [7], # 1
              [7], # 1
             ]
            )

# skew is 0, can use MinMaxScaler
# print(skew(a= X,
#            axis=0))

# change to degree=2 to get lower recall_score
polyer = PolynomialFeatures(degree=3).fit(X= X)

X_polyed = polyer.transform(X= X)

scaler = MinMaxScaler().fit(X= X_polyed)

X_scaled = scaler.transform(X= X_polyed)

#             X1
y = [
     0,0,0, # 0
     1,1,1, # 1
     0,0,0, # 2
     1,1,1, # 3
     0,0,0, # 4
     1,1,1, # 5
     0,0,0, # 6
     1,1,1, # 7
    ]

clf = LogisticRegression()
clf.fit(X= X_scaled, y= y)

# seen data
# X_test = np.array([[4],
#                    [4],
#                    [4],
#                    [5],
#                    [5],
#                    [5]])

# never seen data
X_test = np.array([[8],
                   [8],
                   [8],
                   [9],
                   [9],
                   [9]])

y_test = np.array([0,
                   0,
                   0,
                   1,
                   1,
                   1])

X_test_polyed = polyer.transform(X= X_test)

X_test_scaled = scaler.transform(X= X_test_polyed)

y_test_pred = clf.predict(X= X_test_scaled)

print("y_true", y_test)
print("y_pred", y_test_pred)

# average=None is important.
# otherwise you get average='binary', pos_label=1
print("accuracy score", metrics.accuracy_score(y_true=y_test, y_pred=y_test_pred))
print("recall score", metrics.recall_score(y_true=y_test, y_pred=y_test_pred, average=None))
print("precision score", metrics.precision_score(y_true=y_test, y_pred=y_test_pred, average=None, zero_division=0))
print("f1 score", metrics.f1_score(y_true=y_test, y_pred=y_test_pred, average=None))


y_true [0 0 0 1 1 1]
y_pred [1 1 1 1 1 1]
accuracy score 0.5
recall score [0. 1.]
precision score [0.  0.5]
f1 score [0.         0.66666667]
